In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the Generator model
def build_generator(input_shape):
    model = models.Sequential()
    model.add(layers.Dense(128 * 7 * 7, activation="relu", input_dim=latent_dim))
    model.add(layers.Reshape((7, 7, 128)))
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", activation="relu"))
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", activation="relu"))
    model.add(layers.Conv2D(3, kernel_size=7, activation="sigmoid", padding="same"))
    return model

# Define the Discriminator model
def build_discriminator(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, input_shape=input_shape, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Define the GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = layers.Input(shape=(latent_dim,))
    gan_output = discriminator(generator(gan_input))
    gan = models.Model(gan_input, gan_output)
    return gan

# Load and preprocess automotive images
def load_automotive_images(file_paths):
    # Load images and preprocess (e.g., normalization)
    return preprocessed_images

# Add noise to automotive images
def add_noise(images, noise_factor):
    noisy_images = images + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=images.shape)
    noisy_images = np.clip(noisy_images, 0., 1.)  # Clip values to [0, 1]
    return noisy_images

# Define training parameters
epochs = 100
batch_size = 32
noise_factor = 0.5
latent_dim = 100

# Build and compile the discriminator
input_shape = (128, 128, 3)  # Adjust this according to the size of your input images
discriminator = build_discriminator(input_shape)
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

# Build the generator
generator = build_generator(input_shape)

# Build and compile the GAN model
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Load and preprocess automotive images
file_paths = [...]  # List of file paths to automotive images
images = load_automotive_images(file_paths)

# Add noise to the images
noisy_images = add_noise(images, noise_factor)

# Training loop
for epoch in range(epochs):
    # Select a random batch of images
    idx = np.random.randint(0, images.shape[0], batch_size)
    real_images = images[idx]

    # Generate a batch of noisy images
    idx = np.random.randint(0, noisy_images.shape[0], batch_size)
    noisy_batch = noisy_images[idx]

    # Generate fake images from noisy images
    generated_images = generator.predict(noisy_batch)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(real_images, np.ones(batch_size))
    d_loss_fake = discriminator.train_on_batch(generated_images, np.zeros(batch_size))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # Train the generator (via the GAN model)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.array([1] * batch_size)
    g_loss = gan.train_on_batch(noise, valid_labels)

    # Print progress
    print(f"Epoch {epoch}, D Loss: {d_loss}, G Loss: {g_loss}")

# After training, you can use the generator to denoise new images
denoised_images = generator.predict(new_noisy_images)
